                Directing efforts to improve composting practices in Canada.



Introduction:

Of the 12.9 million tonnes of waste produced by Canadians annually, 8.5 million are sent to waste disposal facilities while 4.4 million are recycled or composted by waste diversion programs. Composting allows food waste to be decomposed aerobically generating carbon dioxide. Food waste that decomposes anaerobically in landfills generates methane, a much more potent greenhouse gas that traps 25 times as much heat as CO2. 

Considering the critical state of Earth’s climate and the dire need to bring net greenhouse gas emissions to zero in the next thirty years as outlined by the IPCC report in October 2018, humans will need to take every available measure to reach that target.

Not only does composting reduce the amount of waste sent to landfills and the greenhouse gases emitted by waste management facilities, but the compost product can be highly valuable as fertilizer. The use of synthetic fertilizer has increased dramatically with industrial agriculture and the depletion of soil quality from climate change and monocultures in particular. Using compost as fertilizer is advantageous in the quest to reduce greenhouse gas emissions by eliminating those released from the production of synthetic fertilizer, but it has also been demonstrated to boost soil quality immensely. Synthetic fertilizers have been linked to severe degradation of the aquatic ecosystems into which they leach, causing eutrophication and subsequent anoxic dead zones. The addition of trace minerals and organic carbon to soils with compost increases their ability to sequester carbon dioxide and increases its structural stability which decreases erosion and harmful leachate into watersheds.

Composting participation in Canada has risen substantially over the last two decades, particularly in municipalities that have developed composting programs such as curb side pickup, waste management centres and wet/dry waste separation. Curbside collection services were most widely used in Census metropolitan areas, where they are most widely available. However, a variety of options exist for composting in urban and rural areas, and the practicality of each one depends primarily on volume of organic waste, space availability, and income. 

The most prominent issues people find with composting are related to limited space and the attraction of pests. A 2011 national census found that composting practices were directly related to dwelling type. Income was also shown to have a large influence on composting behavior, as well as education and tenure of the property. 

Composting rates in single dwellings were over two times higher than those in apartments. In order to increase composting participation, efforts should be targeted to creating programs for demographics demonstrating the lowest participation. This analysis examines whether provinces with poor composting participation have lower proportions of single dwellings, which might indicate a need for composting programs tailored to facilitating composting for apartment dwellers.



Methods:

In [7]:
#load all libraries needed for data analysis
library(rvest)
library(dplyr)
library(ggplot2)

Warning message:
“S3 methods ‘[.fun_list’, ‘[.grouped_df’, ‘all.equal.tbl_df’, ‘anti_join.data.frame’, ‘anti_join.tbl_df’, ‘arrange.data.frame’, ‘arrange.default’, ‘arrange.grouped_df’, ‘arrange.tbl_df’, ‘arrange_.data.frame’, ‘arrange_.tbl_df’, ‘as.data.frame.grouped_df’, ‘as.data.frame.rowwise_df’, ‘as.data.frame.tbl_cube’, ‘as.data.frame.tbl_df’, ‘as.table.tbl_cube’, ‘as.tbl.data.frame’, ‘as.tbl.tbl’, ‘as.tbl_cube.array’, ‘as.tbl_cube.data.frame’, ‘as.tbl_cube.matrix’, ‘as.tbl_cube.table’, ‘as_data_frame.grouped_df’, ‘as_data_frame.tbl_cube’, ‘auto_copy.tbl_cube’, ‘auto_copy.tbl_df’, ‘cbind.grouped_df’, ‘collapse.data.frame’, ‘collect.data.frame’, ‘common_by.NULL’, ‘common_by.character’, ‘common_by.default’, ‘common_by.list’, ‘compute.data.frame’, ‘copy_to.DBIConnection’, ‘copy_to.src_local’, ‘default_missing.data.frame’, ‘default_missing.default’, ‘dim.tbl_cube’, ‘distinct.data.frame’, ‘distinct.default’, ‘distinct.grouped_df’, ‘distinct.tbl_df’, ‘distinct_.data.frame’, ‘distinct_.

ERROR: Error: package or namespace load failed for ‘dplyr’ in library.dynam(lib, package, package.lib):
 shared object ‘dplyr.so’ not found


1. Object was created for Statistics Canada's "Composting by households in Canada" web page where regional composting data was extracted from.

In [8]:
 HoCompost <- read_html("https://www150.statcan.gc.ca/n1/pub/16-002-x/2013001/article/11848-eng.htm")

2. Created object (Messy) for the table containing regional composting data by selecting all the tables on the webpage then isolate the second one using positional indexing in double brackets. The object was then specified to be formatted as a table.

In [9]:
Messy <- HoCompost %>% html_nodes('table')  %>% .[[2]] %>% html_table(fill=TRUE)

3. New object (prov.cdata) was created from the regional data table to by selecting only the column for province and total per cent of households that composted. Then rename the columns

In [10]:
prov.cdata <- Messy[c(4:13),c(1,2)]
colnames(prov.cdata) <- c("Province","Per_cent_composters")
prov.cdata

,Province,Per_cent_composters
4,Newfoundland and Labrador,43
5,Prince Edward Island,96
6,Nova Scotia,94
7,New Brunswick,58
8,Quebec,42
9,Ontario,75
10,Manitoba,56
11,Saskatchewan,47
12,Alberta,56
13,British Columbia,64


4. Created object (Promes) for the Statistics Canada webpage "Type of Dwelling Highlight Tables, 2016 Census" then select the first table on that page which showed the number of dwellings of each type for each province in 2016. The object was refined to only include the 10 provinces and dwelling types that fit the houses (single dwellings) or apartment categories, ie. moveable homes and total were excluded. Appropriate column names were added, then a for loop was used to remove the commas from all the table values so they could then be converted to class numeric which would allow them to be counted and plotted later.


In [11]:
Promes <- read_html("https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/td-tl/Table.cfm?Lang=Eng&T=101&SR=1&S=1&O=A&RPP=25")
Promes <- Promes %>% html_nodes('table')  %>% .[[1]] %>% html_table(fill=TRUE)

Promes <- Promes[c(4:13),c(1,3:9)]
colnames(Promes)<- c("Province","Single_detached_house","Apartment_five_or_more_storeys",
                     "Apartment_under_five_storeys",
                    "Row_house","Semi_detached house","Flat_duplex","Other_single_attached_house" )


for (i in 2:8){Promes[,i] <- as.numeric(gsub(",","",Promes[,i]))}


5. Dwelling types in Promes were categorized to apartments (apartments with more than five storeys and apartments 
   with less than five storeys) and houses (Apartment or flat in a duplex, Row house, Semi-detached house, Other single-attached house, and Single-detached house) by creating vectors of the sum of houses in each of those columns for each province.
 
 The vectors were then made into a dataframe (Provdwell) which also included the names of the Provinces.
 


In [12]:
apartments <- rowSums(Promes[,c(3,4)])
houses <- rowSums(Promes[,c(2,5:8)])

Provdwell<-data.frame("Province"= Promes$Province,"Buildings"= apartments,"Houses" = houses)



6. A new column was added to the Provdwell dataframe to show proportion of Houses in each province 

In [13]:
Provdwell$Prop.houses <- Provdwell$Houses/rowSums(Provdwell[,c(2:3)])
Provdwell

,Province,Buildings,Houses,Prop.houses
4,Newfoundland and Labrador,12390,204860,0.9429689
5,Prince Edward Island,9105,47845,0.8401229
6,Nova Scotia,79755,307360,0.7939760
7,New Brunswick,48210,257775,0.8424433
8,Quebec,1342750,2165410,0.6172495
9,Ontario,1409520,3744760,0.7265341
10,Manitoba,106285,372335,0.7779345
11,Saskatchewan,67640,355660,0.8402079
12,Alberta,285755,1193945,0.8068832
13,British Columbia,562965,1269720,0.6928196


7. Object (final) was created to join the composting and dwelling data by Province using left_join

In [14]:
final<-left_join(Provdwell, prov.cdata)

ERROR: Error in left_join(Provdwell, prov.cdata): could not find function "left_join"


8. Scatterplot was created with the proportion of houses on the x axis and the proportion of composted on the y axis using ggplot2. Labels were added to the axis and colours were used to identify individual provinces

In [15]:
ggplot(final, aes(Prop.houses,Per_cent_composters, col=Province))+
  labs(x="% House Dwellers", y="% Composters", caption="Figure 1 Proportion of houses vs Proportion of composters by province in Canada.")+
  geom_point()

ERROR: Error in ggplot(final, aes(Prop.houses, Per_cent_composters, col = Province)): could not find function "ggplot"


A secondary analysis was pursued to assess whether per capita GDP, a proxy for household income could explain the composting participation for provinces that deviated from a positive linear relationship between proportion of houses and proportion of composters. Data for provincial per capita GDP was extracted from a compiled table on wikepedia, where it was sourced from Statistics Canada.

9. An object (GDP) was created for the wikipedia webpage "List of Canadian provinces and territories by gross domestic product". It was then refined to only include the first table on the page showing Provincial GDP statistics

In [16]:
GDP<-read_html("https://en.wikipedia.org/wiki/List_of_Canadian_provinces_and_territories_by_gross_domestic_product#GDP_and_per_capita_GDP")
GDP <- GDP %>% html_nodes('table')  %>% .[[1]] %>% html_table(fill=TRUE)


10. A for loop was used to remove the coding and '♠' symbol that appeared before all the GDP data values

In [17]:
for(i in 2:11){GDP[i,5]<-strsplit(GDP[,5],'♠')[[i]][2]}

11. the commas were removed from the data values using gsub so they could be converted to class numeric 

In [19]:
GDP[,5]<-as.numeric(gsub(",","",GDP[,5])) 

12. Relevent rows and columns were selected from the original table and renamed appropriately

In [20]:
GDP<-GDP[2:11,c(1,5)]
colnames(GDP)<-c("Province","Percap.GDP")

13. Per capita GDP was then added to the 'final' dataframe using left_join so it could then be plotted against 
 provincial composting participation 


In [21]:
final<-left_join(GDP,final)

ggplot(final, aes(Percap.GDP,Per_cent_composters, col=Province,))+
  geom_point()+
  labs(x="Per Capita GDP($)", y="% Composters",caption="Figure 2. Per capita GDP against Proportion of composters for each province in Canada.")


ERROR: Error in left_join(GDP, final): could not find function "left_join"


Discussion

Plotting proportion of single dwellings to composting participation for each province showed a positive linear relationship for Nova Scotia, Prince Edward Island, Ontario, British Columbia and Quebec. Since British Columbia and Quebec had relatively low proportions of single dwelling houses and composters, efforts to increase composting participation in those provinces might benefit from focusing on developing programs for apartment dwellers.
 The other five provinces had relatively high proportions of single dwellings yet had low composting participation. Those deviances from the positive linear trend do not support Statistics Canada’s conclusion that a direct relationship exists between dwelling type and composting participation. They indicate that factors other than dwelling type have more influence over composting behavior in those provinces. 

Statistics Canada identified income as the second most influential factor on composting participation. When per capita GDP was used as a proxy for household income for each province and plotted against composting activities, Quebec, New Brunswick, Manitoba, British Columbia and Ontario seemed to exhibit a positive linear relationship between them. The apparent relationship may indicate that income is more influential for composting participation in those provinces, particularly in New Brunswick and Manitoba that did not appear to be influenced by proportion of single dwelling types. In those provinces efforts to improve composting participation may also consider focusing on making it more accessible to low income families.

Nova Scotia and Prince Edward Island exhibited the greatest composting activity, while Quebec and Newfoundland composted the least. The stringency of waste management regulations in those provinces have been proposed to explain the differences. The stringency of management regulations was not measured by stats Canada, but that would likely be a very influential factor for composting participation across all provinces because laws work. 

This study had a number of limitations that MAY have affected the integrity of the results. Firstly, the years in which dwelling type data, composting data and GDP data were collected differed by seven years. Secondly, per capita GDP was not the most accurate proxy for household income, but due to limited time and skills of the analyst, the Stats Canada table for household income was not feasible for use in this study. Thirdly, the analysis could have been more precise and informative with a municipal scope rather than provincial, since composting programs tend to operate on municipalities. Finally, positive linear relationships are weak evidence for causal relationships. More in-depth regression analyses would be needed to give validity to this study.

As a signatory of the Paris agreement, Canada has a leading role to play in reduction of greenhouse gases. It is therefore necessary that all efforts be taken to optimize waste management and composting practices. Household composting allows civilians to do their part to reduce their carbon footprint, and composting is a daily reminder to protect the planet.


